# Webscrape data


### Imports and functions

In [1]:
import pandas as pd
import os
from pathlib import Path
import time
import requests
from random import uniform
from bs4 import BeautifulSoup
import random
from fake_useragent import UserAgent
import unicodedata

def remove_special_characters(text):
    # Normalize the string to NFKD form
    normalized = unicodedata.normalize('NFKD', text)
    # Remove diacritics (non-spacing marks)
    without_special_chars = ''.join(c for c in normalized if not unicodedata.combining(c))
    return without_special_chars

### Naming convention for the tables

In [2]:
titles_12 = [
    'serie_A_overall',
    'serie_A_homeaway',
    'squad_standard_stats',
    'squad_standard_stats_opp',
    'squad_goalkeeping',
    'squad_goalkeeping_opp',
    'squad_advanced_goalkeeping',
    'squad_advanced_goalkeeping_opp',
    'squad_shooting',
    'squad_shooting_opp',
    'squad_passing',
    'squad_passing_opp',
    'squad_pass_types',
    'squad_pass_types_opp',
    'squad_goal_shot_creation',
    'squad_goal_shot_creation_opp',
    'squad_defensive_actions',
    'squad_defensive_actions_opp',
    'squad_possession',
    'squad_possession_opp',
    'squad_playing_time',
    'squad_playing_time_opp',
    'squad_miscellaneous',
    'squad_miscellaneous_opp',
]

titles_6 = titles_12[:6] + titles_12[8:10] + titles_12[20:]

titles_5 = titles_6[:6] + titles_6[8:]

### Acquire the league data

In [ ]:
os.makedirs('data/clubes/', exist_ok=True)
os.makedirs('data/league/', exist_ok=True)

In [ ]:
# Fetch all tables from the webpage
os.makedirs('data', exist_ok=True)
for page in range(11):
    
    url = f"https://fbref.com/en/comps/24/{2024 - page}/{2024 - page}-Serie-A-Stats"
    tables = pd.read_html(url)
    year = 2024 - page
    
    # Convert each table to a DataFrame
    if year > 2018:
        dfs = {f"{titles_12[i]}_{year}": table for i, table in enumerate(tables)}
    
    elif year > 2015:
        dfs = {f"{titles_6[i]}_{year}": table for i, table in enumerate(tables)}
    
    else:
        dfs = {f"{titles_5[i]}_{year}": table for i, table in enumerate(tables)}
    
    for name, df in dfs.items():
        df.to_csv(f"data/league/{name}.csv", index=False)


### Acquire player data

#### Get squads that were on Serie A and the year

In [1]:
## ID da página de acesso de cada clube
clubes = {
        'Botafogo-RJ': {'id': 'd9fdd9d9'},
        'Palmeiras': {'id': 'abdce579'},
        'Flamengo': {'id': '639950ae'},
        'Fortaleza': {'id': 'a9d0ab0e'},
        'Internacional': {'id': '6f7e1f03'},
        'Sao-Paulo': {'id': '5f232eb1'},
        'Corinthians': {'id': 'bf4acd28'},
        'Bahia': {'id': '157b7fee'},
        'Cruzeiro': {'id': '03ff5eeb'},
        'Vasco-da-Gama': {'id': '83f55dbe'},
        'Vitoria': {'id': '33f95fe0'},
        'Atletico-Mineiro': {'id': '422bb734'},
        'Fluminense': {'id': '84d9701c'},
        'Gremio': {'id': 'd5ae3703'},
        'Juventude': {'id': 'd081b697'},
        'RB-Bragantino': {'id': 'f98930d1'},
        'Ath-Paranaense': {'id': '2091c619'},
        'Criciuma': {'id': '3f7595bb'},
        'Atl-Goianiense': {'id': '32d508ca'},
        'Cuiaba': {'id': 'f0e6fb14'},
        'Santos': {'id': '712c528f'},
        'Goias': {'id': '78c617cc'},
        'Coritiba': {'id': 'd680d257'},
        'America-MG': {'id': '1f68d780'},
        'Ceara': {'id': '2f335e17'},
        'Avai': {'id': 'f205258a'},
        'Sport-Recife': {'id': 'ece66b78'},
        'Chapecoense': {'id': 'baa296ad'},
        'CSA': {'id': '05aff519'},
        'Parana': {'id': '2091c619'},
        'Ponte-Preta': {'id': 'b162ebe7'},
        'Figueirense': {'id': '0ce4436d'},
        'Santa-Cruz': {'id': 'ad0c1246'},
        'Joinville': {'id': 'da0666a2'},
    }
CLUBES = clubes.keys()

In [3]:
for clube in clubes:
    clubes[clube]['years_SA'] = []

In [4]:
# verifica se o time estava na serie A naquele ano e adiciona o ano na key 'years_SA'

for i in range(11):
    df = pd.read_csv(f'data/league/serie_A_overall_{2024 - i}.csv')

    unique_values = df['Squad'].unique()

    for value in unique_values:
        value = remove_special_characters(value.replace(" ", "-").replace("(", "").replace(")", ""))

        clubes[value]['years_SA'].append(2024 - i)
        

#### Scraping with pd.read_html()

In [ ]:
for clube, dados in clubes.items():
    # Create club directory if it doesn't exist
    club_dir = Path(f'data/clubes/{clube}')
    club_dir.mkdir(parents=True, exist_ok=True)
    
    for year in dados['years_SA']:
        # Generate expected filenames pattern for this year
        existing_files = list(club_dir.glob(f"{year}_*.csv"))
        
        # Skip if we already have files for this year
        if existing_files:
            print(f"Skipping {clube} {year} - already downloaded")
            continue
        
        url = f"https://fbref.com/en/squads/{clubes[clube]['id']}/{year}/{clube}-Stats"

        try:
            # Add delay to avoid 429 errors
            
            time.sleep(uniform(5, 30))
            tables = pd.read_html(url)
            
            # Save each table
            for i, table in enumerate(tables):
                filename = club_dir / f"{year}_{i}.csv"
                table.to_csv(filename, index=False)
            
            print(f"Saved {clube} {year}")
                
        except Exception as e:
            print(f"Failed to scrape {clube} {year}: {str(e)}")
            continue


Skipping Botafogo-RJ 2024 - already downloaded
Skipping Botafogo-RJ 2023 - already downloaded
Skipping Botafogo-RJ 2022 - already downloaded
Skipping Botafogo-RJ 2020 - already downloaded
Skipping Botafogo-RJ 2019 - already downloaded
Skipping Botafogo-RJ 2018 - already downloaded
Skipping Botafogo-RJ 2017 - already downloaded
Skipping Botafogo-RJ 2016 - already downloaded
Skipping Botafogo-RJ 2014 - already downloaded
Skipping Palmeiras 2024 - already downloaded
Skipping Palmeiras 2023 - already downloaded
Skipping Palmeiras 2022 - already downloaded
Skipping Palmeiras 2021 - already downloaded
Skipping Palmeiras 2020 - already downloaded
Skipping Palmeiras 2019 - already downloaded
Skipping Palmeiras 2018 - already downloaded
Skipping Palmeiras 2017 - already downloaded
Skipping Palmeiras 2016 - already downloaded
Skipping Palmeiras 2015 - already downloaded
Skipping Palmeiras 2014 - already downloaded
Skipping Flamengo 2024 - already downloaded
Skipping Flamengo 2023 - already downl

#### Rename the tables

In [17]:
title_clubes_13 = [
    'standard',
    'scores_fixtures',
    'goalkeeping',
    'advanced_goalkeeping',
    'shooting',
    'passing',
    'pass_types',
    'goal_shot_creation',
    'defesensive_actions',
    'possession',
    'playingtime',
    'miscellaneous',
    'serie_A_overall',
    'serie_A_homeaway',
]
title_clubes_7 = title_clubes_13[:3] + [title_clubes_13[4]] + title_clubes_13[10:]


In [ ]:
for clube, dados in clubes.items():
    
    club_dir = Path(f'data/clubes/{clube}')
    
    files = os.listdir(club_dir)
    
    for file in files:
       
        if file.split('_')[-1].split('.')[0].isdigit():
            
            year, digit = file.split('_')
            digit = digit.split('.')[0]

            if int(year) > 2018:
                new_name = f"{year}_{title_clubes_13[int(digit)]}.csv"
            else:
                new_name = f"{year}_{title_clubes_7[int(digit)]}.csv"


            os.rename(club_dir / file, club_dir / new_name)
            #print(club_dir / file, club_dir / new_name)




#### Test scraping

In [10]:
url = f"https://fbref.com/en/squads/d9fdd9d9/2022/Botafogo-RJ-Stats"
    

tables = pd.read_html(url)

# Save each table
for i, table in enumerate(tables):
    filename = f"data/clubes/Botafogo-RJ/{2022}_{i}.csv"
    table.to_csv(filename, index=False)

# Preprocessing

### Imports and functions

In [6]:
import pandas as pd
import os
from pathlib import Path

clubes = { # site id for each squad
        'Botafogo-RJ': {'id': 'd9fdd9d9'},
        'Palmeiras': {'id': 'abdce579'},
        'Flamengo': {'id': '639950ae'},
        'Fortaleza': {'id': 'a9d0ab0e'},
        'Internacional': {'id': '6f7e1f03'},
        'Sao-Paulo': {'id': '5f232eb1'},
        'Corinthians': {'id': 'bf4acd28'},
        'Bahia': {'id': '157b7fee'},
        'Cruzeiro': {'id': '03ff5eeb'},
        'Vasco-da-Gama': {'id': '83f55dbe'},
        'Vitoria': {'id': '33f95fe0'},
        'Atletico-Mineiro': {'id': '422bb734'},
        'Fluminense': {'id': '84d9701c'},
        'Gremio': {'id': 'd5ae3703'},
        'Juventude': {'id': 'd081b697'},
        'RB-Bragantino': {'id': 'f98930d1'},
        'Ath-Paranaense': {'id': '2091c619'},
        'Criciuma': {'id': '3f7595bb'},
        'Atl-Goianiense': {'id': '32d508ca'},
        'Cuiaba': {'id': 'f0e6fb14'},
        'Santos': {'id': '712c528f'},
        'Goias': {'id': '78c617cc'},
        'Coritiba': {'id': 'd680d257'},
        'America-MG': {'id': '1f68d780'},
        'Ceara': {'id': '2f335e17'},
        'Avai': {'id': 'f205258a'},
        'Sport-Recife': {'id': 'ece66b78'},
        'Chapecoense': {'id': 'baa296ad'},
        'CSA': {'id': '05aff519'},
        'Parana': {'id': '2091c619'},
        'Ponte-Preta': {'id': 'b162ebe7'},
        'Figueirense': {'id': '0ce4436d'},
        'Santa-Cruz': {'id': 'ad0c1246'},
        'Joinville': {'id': 'da0666a2'},
}

CLUBES = clubes.keys()

def process_names(df, multi_index=False, drop_matches = False):

    """
    Renames columns in the DataFrame based on their first row values,
    drops the first row, removes the 'Matches' column if it exists, 
    and resets the index.

    Parameters:
    df (pd.DataFrame): The input DataFrame to process.

    Returns:
    pd.DataFrame: The processed DataFrame with updated column names, 
                  the first row removed, and the index reset.
    """

    # Create rename dictionary first to avoid modifying DataFrame during iteration
    rename_dict = {}
    for col in df.columns:
        # Use .at[] for faster scalar value access
        first_row_value = df.at[0, col]
        
        if 'Unnamed' in col:
            # set the new column name depending on whether multi-index is True
            new_name = f"Info.{first_row_value}" if multi_index else first_row_value
        
        else:
            # set the new column name depending on whether multi-index is True
            prefix = col.split('.')[0]
            new_name = f"{prefix}.{first_row_value}" if multi_index else f"{prefix} {first_row_value}"
        
        rename_dict[col] = new_name

    # Apply all renames at once
    df = df.rename(columns=rename_dict)
    
    # Drop first row (index 0)
    df = df.drop(index=0)
    
    if drop_matches:
        # Set matches column name
        matches = 'Info.Matches' if multi_index else 'Matches'
        
        # Safely drop 'Matches' column if it exists
        if matches in df.columns:
            df = df.drop(columns=[matches])
    
    if multi_index:
        # set multi-index column names
        df.columns = pd.MultiIndex.from_tuples([tuple(col.split('.')) for col in df.columns])
    
    # Reset index and return the processed df
    return df.reset_index(drop=True)

def get_gringos(df, gringos, clube=None, year=None, nation=['br BRA']):

    """
    Filters the provided dataframe and returns only the players 
    that are not from the especified nationality(s).

    Parameters:
    df (pd.DataFrame): The input DataFrame to filter and process.
    gringos (pd.DataFrame): The existing DataFrame to append filtered rows to.
    clube (str, optional): Pass a name to add the squad name to the new df.
    nation (list): The nation(s) to be filtered out, default is 'br BRA'.

    Returns:
    pd.DataFrame: The updated `gringos` DataFrame with filtered rows 
                  appended and the optional 'Clube' and 'Year' columns added.
    """

    # Create a copy of the filtered data to avoid SettingWithCopyWarning
    new_gringos = df[~df['Nation'].isin(nation) & ~df['Nation'].isna()].copy()
    
    if clube:
        # Add column with squad names on position 5
        new_gringos.insert(loc=5, column='Clube', value=clube)
        #new_gringos['Clube'] = clube
    
    if year:
        # Add column with year on position 6
        new_gringos.insert(loc=6, column='Year', value=year)
        #new_gringos['Year'] = year

    if gringos.empty:
        gringos = new_gringos

    else:
        gringos = pd.concat([gringos, new_gringos], ignore_index=True)
    
    return gringos

### Get foreigners from all datasets and save into .csv

In [9]:
gringos_df = pd.DataFrame()

for clube in CLUBES:
    
    club_dir = Path(f'data/clubes/{clube}')
    
    files = os.listdir(club_dir)
    
    for file in files:
        year = file.split('_')[0]
        if "standard" in file: #and year == '2024':
            temp_df = process_names(pd.read_csv(f'{club_dir}/{file}'))
            gringos_df = get_gringos(temp_df, gringos_df, clube=clube, year=year)

gringos_df.to_csv('data/processed/gringos_df.csv', index=False)

In [10]:
gringos_df

,Player,Nation,Pos,Age,MP,Clube,Year,Playing Time Starts,Playing Time Min,Playing Time 90s,...,Expected npxG+xAG,Progression PrgC,Progression PrgP,Progression PrgR,Per 90 Minutes xG,Per 90 Minutes xAG,Per 90 Minutes xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG,Playing Time MP
0,Pablo Zeballos,py PAR,"FW,MF",27.0,30,Botafogo-RJ,2014,22,1907.0,21.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mario Bolatti,ar ARG,MF,28.0,26,Botafogo-RJ,2014,19,1586.0,17.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Emerson Sheik,qa QAT,FW,35.0,15,Botafogo-RJ,2014,15,1318.0,14.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Luis Alberto Ramírez,pe PER,"FW,MF",29.0,16,Botafogo-RJ,2014,14,1270.0,14.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Juan Carlos Ferreyra,ar ARG,FW,30.0,10,Botafogo-RJ,2014,6,569.0,6.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,Michael Ortega,co COL,MF,24.0,NaN,Figueirense,2016,0,142.0,1.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1016,Matías Pisano,ar ARG,"FW,MF",24.0,11,Santa-Cruz,2016,8,575.0,6.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1017,Álex Bolaños,ec ECU,MF,31.0,3,Santa-Cruz,2016,2,113.0,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,Gutieri Tomelin,it ITA,DF,23.0,34,Joinville,2015,33,2908.0,32.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Analysis

## Load foreigner df

In [1]:
import pandas as pd

In [11]:
gringos = pd.read_csv('data/processed/gringos_df.csv')